In [90]:
from dataset_loader import *
from classifiers import *
from result_utils import *
from sklearn.cluster import DBSCAN, KMeans
import numpy as np
from tqdm import tqdm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from classifiers import *
from result_utils import *

In [91]:
dataset_name = 'WESAD_WRIST'
dataset_name = 'AffectiveROAD'

In [92]:
WINDOW_SIZE = 60
WINDOW_SHIFT = 1

In [93]:
SCORING = 'balanced_accuracy'

In [94]:
ds_loader = DatasetLoader(dataset_name, WINDOW_SIZE = WINDOW_SIZE, WINDOW_SHIFT = WINDOW_SHIFT)
result_helper = ResultUtils(dataset_name, WINDOW_SIZE = WINDOW_SIZE, WINDOW_SHIFT = WINDOW_SHIFT)

In [95]:
detection_strategy = 'random_forest'

In [96]:
X = ds_loader.dataset
y = ds_loader.ground_truth
groups = ds_loader.groups

In [100]:
logo = LeaveOneGroupOut()
new_X = np.array([])
new_y = np.array([])
new_groups = np.array([])
for train_index, test_index in logo.split(X, y, groups):
    _X, _y = X[test_index], y[test_index]
    model = KMeans(n_clusters = 3, random_state = 0)
    model.fit(_X)
    scores = []
    for index in list(set(model.labels_)):
        indices = np.where(model.labels_ == index)[0]
        __y = _y[indices]
        scores.append(np.median(__y))
    sorted_indices = np.argsort(scores)
    print(scores, np.argsort(scores))
    for label, index in enumerate(sorted_indices):
        if label == 1: continue
        indices = np.where(model.labels_ == index)[0]
        _y[indices] = min(label, 1)
        new_y = np.concatenate((new_y, _y[indices]))
        if new_X.shape[0] == 0:
            new_X = _X[indices]
        else:
            new_X = np.concatenate((new_X, _X[indices]))
        new_groups = np.concatenate((new_groups, groups[test_index][indices]))

[0.2711110738689664, 0.6776262865250516, 0.0] [2 0 1]
[0.5503061727260281, 0.0, 0.0] [1 2 0]
[0.0, 0.5398219773779167, 0.8067961343988724] [0 1 2]
[0.6214357301699539, 0.57722047239123, 0.0] [2 1 0]
[0.46801309504151334, 0.41825492273625536, 0.0] [2 1 0]
[0.23621440713858732, 0.0, 0.5503121185725278] [1 0 2]
[0.6130810394333873, 0.0, 0.0] [1 2 0]
[0.5548419709809188, 0.8138550836845277, 0.8194364198348578] [0 1 2]
[0.849613576183839, 0.0, 0.7408680594919594] [1 2 0]
[0.6521027256847292, 0.0, 0.0] [1 2 0]
[0.0, 0.8596233378590816, 0.5515039419238178] [0 2 1]
[0.5577855370899276, 0.49241996924738063, 0.0] [2 1 0]
[0.41137500000000005, 0.0, 0.4940616666666667] [1 0 2]


In [101]:
print(set(new_groups))

{'Drv13', 'Drv6', 'Drv5', 'Drv1', 'Drv12', 'Drv4', 'Drv11', 'Drv2', 'Drv3', 'Drv10', 'Drv8', 'Drv7', 'Drv9'}


In [102]:
strategies = ['random_forest']
SCORING = 'balanced_accuracy'
for detection_strategy in strategies:
    detector_type = 'General'
    print(f'--- RUNNING {detector_type} {detection_strategy} ---')
    clf = BinaryClassifier(new_X, new_y, detection_strategy, logo_validation = True, groups = new_groups, scoring = SCORING)
    # clf = BinaryClassifier(ds_loader.hrv_dataset, ds_loader.ground_truth, detection_strategy, logo_validation = True, groups = ds_loader.groups, scoring = SCORING)
    # clf = BinaryClassifier(ds_loader.eda_dataset, ds_loader.ground_truth, detection_strategy, logo_validation = True, groups = ds_loader.groups, scoring = SCORING)
    results = clf.exec_classifier() # Build classifier and return prediction results
    print('------------------------------------------------------')

    # # %%
    # # Save results
    result_helper = ResultUtils(dataset_name, WINDOW_SHIFT = WINDOW_SHIFT, WINDOW_SIZE = WINDOW_SIZE)
    result_helper.dump_result_to_csv(results, detection_strategy, detector_type, physiological_signal_type = '')

0it [00:00, ?it/s]

--- RUNNING General random_forest ---
Counter({0.0: 16571, 1.0: 13214})


1it [00:08,  8.09s/it]

--- Test Group Drv1 BA Score: 0.36384976525821594 --- Train BA Score 1.0 ---
Counter({0.0: 15428, 1.0: 12043})


2it [00:15,  7.90s/it]

...

--- Test Group Drv11 BA Score: 0.4332411245878209 --- Train BA Score 1.0 ---
Counter({0.0: 15851, 1.0: 11807})


4it [00:30,  7.71s/it]

--- Test Group Drv12 BA Score: 0.39421813403416556 --- Train BA Score 1.0 ---
Counter({0.0: 16263, 1.0: 12275})


4it [00:33,  8.42s/it]


KeyboardInterrupt: 

In [ ]:
# logo = LeaveOneGroupOut()
# test_groups = []
# balanced_accs = []
# for train_index, test_index in tqdm(logo.split(X, y, groups)):
#     X_train, y_train, X_test, y_test = X[train_index], y[train_index], X[test_index], y[test_index]
#     print(f'--- RUNNING MODEL CLUSTERING {groups[test_index][0]} ---')
#     scaler = StandardScaler()
#     scaler.fit(X_train)
#     X_train = scaler.transform(X_train)
#     X_test = scaler.transform(X_test)
#     # cluster_model = KMeans(n_clusters = 3, random_state = 0, n_jobs = -1, max_iter = 10000).fit(X_train)
#     print(f'--- TRAINING MODEL {groups[test_index][0]} ---')
        # estimators = [('rf', RandomForestClassifier(n_estimators = 1000, random_state = 0, n_jobs = 8, max_features='sqrt',
        #                                 oob_score=True, bootstrap=True, class_weight = 'balanced', )),    
        #     ('svm', SVC(C = 10, random_state = 0, class_weight = 'balanced')),
        #     ('mlp', MLPClassifier(random_state = 0, early_stopping = True, max_iter = 1000, activation = 'logistic'))
        # ]
 
    # _X_train = cluster_model.transform(X_train)
    # _X_test = cluster_model.transform(X_test)
    # y_preds = vclf.predict(X_test)
    # clf = RandomForestClassifier(n_estimators = 1000, random_state = 0, n_jobs = 8, max_features='sqrt',
                                    # oob_score=True, bootstrap=True, class_weight = 'balanced', )
    # pipeline = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components = 8)), 
    #         ('kmeans', KMeans(n_clusters = 3, random_state = 0, n_jobs = -1, max_iter = 10000)),
    #         ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 0, n_jobs = 8, max_features='sqrt',
    #                                 oob_score=True, bootstrap=True, class_weight = 'balanced', ))
    #     ])
    # pipeline.fit(X_train)
    # y_preds = pipeline.predict(X_test)
    # test_labels = pipeline.predict(X_test)
    # y_preds = y_test.copy()
    # for i in list(set(pipeline['kmeans'].labels_)):
    #     # clf = RandomForestClassifier(n_estimators = 1000, random_state = 0, n_jobs = 8, max_features='sqrt',
    #                                 # oob_score=True, bootstrap=True, class_weight = 'balanced', )    
    #     estimators = [('rf', RandomForestClassifier(n_estimators = 1000, random_state = 0, n_jobs = 8, max_features='sqrt',
    #                                     oob_score=True, bootstrap=True, class_weight = 'balanced', )),    
    #         ('svm', SVC(C = 10, random_state = 0, class_weight = 'balanced')),
    #         ('mlp', MLPClassifier(random_state = 0, early_stopping = True, max_iter = 1000, activation = 'logistic'))
    #     ]
    #     clf = VotingClassifier(estimators = estimators, n_jobs=-1, verbose = True)
    #     clf.fit(X_train, y_train)
    #     # clf = SVC(C = 10, random_state = 0, class_weight = 'balanced')
    #     indices = np.where(pipeline['kmeans'].labels_ == i)[0]
    #     _indices = np.where(test_labels == i)[0]
    #     if len(_indices) == 0:
    #         continue
    #     _X = scaler.fit_transform(X_train[indices])
    #     _X_test = scaler.transform(X_test[_indices])
    #     _y = y_train[indices]
    #     clf.fit(_X, _y)
    #     pred = clf.predict(_X_test)
    #     y_preds[_indices] = pred
    #     # estimators.append((f'rf_{i+1}', clf))
    
    acc = balanced_accuracy_score(y_test, y_preds)
    print(f'Test {groups[test_index[0]]}: {acc}')
    # break
    # clf = BinaryClassifier(X_train, y_train, detection_strategy, scoring = SCORING)
    # acc = clf.train_and_infer(X_test, y_test)
    # balanced_accs.append(acc)
    # test_groups.append(groups[test_index][0])
    # break
# results = { 'groups': test_groups, 'balanced_accuracy_score': balanced_accs }

IndentationError: unexpected indent (<ipython-input-83-fc6c02821b70>, line 56)